In [1]:
import requests
import time
import datetime
import pandas as pd
from config import email, key
import matplotlib.pyplot as plt
import json
import numpy

In [2]:
events = ["ci37908735","ci37374687","nc72282711","ci15200401","ci15199681","ci10736069",
          "ci14383980","nc40216664","nc40204628"]

i=0

earthquakes_list = []

for event in events:
    base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
    times_url = "&starttime=1990-01-01&endtime=2019-06-15"
    magnitude_url ="&minmagnitude=5.0"
    #geometry_url = "&minlatitude=32.43&maxlatitude=42.05&minlongitude=-124.28&maxlongitude=-114.11"
    eventid_url = f"&eventid={event}"
    
    search = requests.get(base_url + times_url + magnitude_url + eventid_url)
    
    response = search.json()

    identifier = response['id']
    mag = response['properties']['mag']
    url = response['properties']['url']
    epochtime = response['properties']['time']
    four_weeks_before = epochtime - 2419200000
    four_weeks_after = epochtime + 2419200000
    event_date= time.strftime('%Y%m%d',  time.gmtime(epochtime/1000))
    month_before= time.strftime('%Y%m%d',  time.gmtime(four_weeks_before/1000))
    month_after = time.strftime('%Y%m%d',  time.gmtime(four_weeks_after/1000))
    converted_time = time.strftime('%H:%M:%S',  time.gmtime(epochtime/1000))
    timezone=response['properties']['tz']
    place = response['properties']['place']
    significance=response['properties']['sig']
    lon = response['geometry']['coordinates'][0]
    lat = response['geometry']['coordinates'][1]
    depth = response['geometry']['coordinates'][2]
    record_dict = {"Identifier":identifier, "Location":place, "Magnitude":mag, "Event Date":event_date,
                   "Month Before":month_before, "Month After":month_after, "Time":converted_time,"Time Zone":timezone, 
                   "Significance":significance, "Lat":lat, "Lon":lon, "Depth":depth, "Epoch Time":epochtime}
    earthquakes_list.append(record_dict)
        
    i += 1
          
earthquakes_df = pd.DataFrame(earthquakes_list)

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Mexico")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("MX")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("Nevada")]

earthquakes_df = earthquakes_df[~earthquakes_df["Location"].str.contains("NV")].reset_index()

earthquakes_df = earthquakes_df.fillna(value="")

earthquakes_abbrev =earthquakes_df[['Location', 'Event Date', 'Magnitude']]

pd.set_option('display.max_rows', None)

earthquakes_df.to_csv("DataFiles/earthquake_list.csv")

earthquakes_df 

,index,Identifier,Location,Magnitude,Event Date,Month Before,Month After,Time,Time Zone,Significance,Lat,Lon,Depth,Epoch Time
0,0,ci37908735,"29km SW of Santa Cruz Is. (E end), CA",5.29,20180405,20180308,20180503,19:29:16,-480,1011,33.837500,-119.725833,9.870,1522956556500
1,1,ci37374687,"20km NNW of Borrego Springs, CA",5.19,20160610,20160513,20160708,08:04:38,-420,1014,33.431500,-116.442667,12.310,1465545878700
2,2,nc72282711,South Napa,6.02,20140824,20140727,20140921,10:20:44,-420,2750,38.215167,-122.312333,11.120,1408875644070
3,3,ci15200401,"4km NNW of Brawley, CA",5.41,20120826,20120729,20120923,20:57:58,-480,1090,33.018500,-115.540333,8.283,1346014678220
4,4,ci15199681,"5km NNW of Brawley, CA",5.32,20120826,20120729,20120923,19:31:23,,1370,33.017167,-115.553667,8.323,1346009483040
5,5,ci10736069,"20km NNW of Borrego Springs, CA",5.42,20100707,20100609,20100804,23:53:33,-480,1032,33.417333,-116.474667,12.318,1278546813480
6,6,ci14383980,"5km S of Chino Hills, CA",5.44,20080729,20080701,20080826,18:42:15,,1095,33.948500,-117.766333,15.503,1217356935670
7,7,nc40216664,Northern California,5.40,20080430,20080402,20080528,03:03:06,,969,40.835833,-123.496833,27.755,1209524586900
8,8,nc40204628,"San Francisco Bay area, California",5.45,20071031,20071003,20071128,03:04:54,,1067,37.433500,-121.774333,9.741,1193799894810


In [18]:
earthquake = 0
before_list = []
after_list = []

for rows in earthquakes_df.iterrows():

    # base_url finds air quality readings by Box with daily summaries 

    base_url = "https://aqs.epa.gov/data/api/dailyData/byBox?"

    #parameters for byBox for CRITERIA Pollutants 
    #42401 = "Sulfur Dioxide" 
    #42602 = "Nitrogen Dioxide"
    #44201 = "Ozone"
    #42101 = "Carbon Monoxide"
    #81102 = "PM10 Total 0-10 um STP"
    #88101 = "PM2.5 - Local Conditions"

    begin = earthquakes_df.iloc[earthquake, 5]
    end = earthquakes_df.iloc[earthquake, 6]
    minlat=earthquakes_df.iloc[earthquake,10]-1
    maxlat=earthquakes_df.iloc[earthquake,10]+1
    minlon=earthquakes_df.iloc[earthquake,11]-1
    maxlon=earthquakes_df.iloc[earthquake,11]+1
    earthquake_location=earthquakes_df.iloc[earthquake,2]
    event_date =earthquakes_df.iloc[earthquake, 4]
    identifier = earthquakes_df.iloc[earthquake,1]
    
    param_url = f"param=42401,42602,44201,42101,81102,88101"
    date_range_url = f"&bdate={begin}&edate={event_date}"
    geometry_url = f"&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}"

    url = (base_url + param_url + "&email=" + email + "&key=" + key + date_range_url + geometry_url)
    
    print(url)
    
    print(earthquake_location+" "+event_date)
    
    search = requests.get(url)
    responses = search.json()
    
    i = 0
    
    for record in responses['Data']:

        parameter = responses['Data'][i]['parameter']
        date = responses['Data'][i]['date_local']
        measurement = responses['Data'][i]['arithmetic_mean']
        earthquake_id = earthquakes_df.iloc[earthquake, 1]
        record_dict = {"Earthquake ID":earthquake_id,"Parameter":parameter,"Date":date,"Measurement":measurement}
        before_list.append(record_dict)

        i += 1
    
    earthquake += 1

print('Searches Completed')

https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20180308&edate=20180405&minlat=32.8375&maxlat=34.8375&minlon=-120.7258333&maxlon=-118.7258333
29km SW of Santa Cruz Is. (E end), CA 20180405
https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20160513&edate=20160610&minlat=32.4315&maxlat=34.4315&minlon=-117.4426667&maxlon=-115.4426667
20km NNW of Borrego Springs, CA 20160610
https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20140727&edate=20140824&minlat=37.2151667&maxlat=39.2151667&minlon=-123.3123333&maxlon=-121.3123333
South Napa 20140824
https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20120729&edate=20120826&minlat=32.0185&maxlat=34.018

In [19]:
before_df = pd.DataFrame(before_list)
pd.set_option('display.max_rows', 100)
before_df

,Earthquake ID,Parameter,Date,Measurement
0,ci37908735,PM10 Total 0-10um STP,2018-03-09,21.571429
1,ci37908735,PM10 Total 0-10um STP,2018-03-10,13.500000
2,ci37908735,PM10 Total 0-10um STP,2018-03-11,14.666667
3,ci37908735,PM10 Total 0-10um STP,2018-03-12,11.583333
4,ci37908735,PM10 Total 0-10um STP,2018-03-13,10.875000
...,...,...,...,...
56574,nc40204628,PM10 Total 0-10um STP,2007-10-21,6.000000
56575,nc40204628,PM10 Total 0-10um STP,2007-10-27,33.000000
56576,nc40204628,Ozone,2007-10-31,0.011000
56577,nc40204628,Ozone,2007-10-31,0.011000


In [20]:
earthquake = 0
before_list = []
after_list = []

for rows in earthquakes_df.iterrows():

    # base_url finds air quality readings by Box with daily summaries 

    base_url = "https://aqs.epa.gov/data/api/dailyData/byBox?"

    #parameters for byBox for CRITERIA Pollutants 
    #42401 = "Sulfur Dioxide" 
    #42602 = "Nitrogen Dioxide"
    #44201 = "Ozone"
    #42101 = "Carbon Monoxide"
    #81102 = "PM10 Total 0-10 um STP"
    #88101 = "PM2.5 - Local Conditions"

    begin = earthquakes_df.iloc[earthquake, 5]
    end = earthquakes_df.iloc[earthquake, 6]
    minlat=earthquakes_df.iloc[earthquake,10]-1
    maxlat=earthquakes_df.iloc[earthquake,10]+1
    minlon=earthquakes_df.iloc[earthquake,11]-1
    maxlon=earthquakes_df.iloc[earthquake,11]+1
    earthquake_location=earthquakes_df.iloc[earthquake,2]
    event_date =earthquakes_df.iloc[earthquake, 4]
    identifier = earthquakes_df.iloc[earthquake,1]
    
    param_url = f"param=42401,42602,44201,42101,81102,88101"
    date_range_url = f"&bdate={event_date}&edate={end}"
    geometry_url = f"&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}"

    url = (base_url + param_url + "&email=" + email + "&key=" + key + date_range_url + geometry_url)
    
    print(url)
    
    print(earthquake_location+" "+event_date)
    
    search = requests.get(url)
    responses = search.json()
    
    i = 0
    
    for record in responses['Data']:

        parameter = responses['Data'][i]['parameter']
        date = responses['Data'][i]['date_local']
        measurement = responses['Data'][i]['arithmetic_mean']
        earthquake_id = earthquakes_df.iloc[earthquake, 1]
        record_dict = {"Earthquake ID":earthquake_id,"Parameter":parameter,"Date":date,"Measurement":measurement}
        after_list.append(record_dict)

        i += 1
    
    earthquake += 1

print('Searches Completed')

https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20180405&edate=20180503&minlat=32.8375&maxlat=34.8375&minlon=-120.7258333&maxlon=-118.7258333
29km SW of Santa Cruz Is. (E end), CA 20180405
https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20160610&edate=20160708&minlat=32.4315&maxlat=34.4315&minlon=-117.4426667&maxlon=-115.4426667
20km NNW of Borrego Springs, CA 20160610
https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20140824&edate=20140921&minlat=37.2151667&maxlat=39.2151667&minlon=-123.3123333&maxlon=-121.3123333
South Napa 20140824
https://aqs.epa.gov/data/api/dailyData/byBox?param=42401,42602,44201,42101,81102,88101&email=ronneimark@hotmail.com&key=tawnybird31&bdate=20120826&edate=20120923&minlat=32.0185&maxlat=34.018

In [21]:
after_df = pd.DataFrame(after_list)
pd.set_option('display.max_rows', 100)
after_df

,Earthquake ID,Parameter,Date,Measurement
0,ci37908735,PM10 Total 0-10um STP,2018-04-15,33.166667
1,ci37908735,PM10 Total 0-10um STP,2018-04-16,21.750000
2,ci37908735,PM10 Total 0-10um STP,2018-04-17,24.708333
3,ci37908735,PM10 Total 0-10um STP,2018-04-18,21.083333
4,ci37908735,PM10 Total 0-10um STP,2018-04-19,21.375000
...,...,...,...,...
57084,nc40204628,PM10 Total 0-10um STP,2007-11-20,25.000000
57085,nc40204628,PM10 Total 0-10um STP,2007-11-26,38.000000
57086,nc40204628,Ozone,2007-10-31,0.011000
57087,nc40204628,Ozone,2007-10-31,0.011000
